In [1]:
import scanpy as sc
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import gseapy
import pandas as pd 
from pathlib import Path
import matplotlib.colors as mcolors
from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
import os
import sys
import hashlib
import colorsys
from graphviz import Digraph
from adjustText import adjust_text
import matplotlib.patheffects as path_effects
import random

sys.path.append('../src/TMEGPT')
from Graph import Node, Graph, Edge
import pickle
from scipy.stats import ttest_ind
import networkx as nx
import xlrd
import time
import requests
import mygene




In [2]:
supplementary_table_1_nature = pd.read_excel('../data/NIHMS982390-supplement-Table_1 (1).xls')

In [3]:
supplementary_table_1_nature.head()

,Sample Name,normallung1,normallung2,CC1324KPT-E,CC1331KPT-E,CC1334KPT-E,DC0179KT,DC0182KT,DC0185KT,CC0299T1,...,CC0560ST,CC0543ST,CC0802ST1,CC1032ST2,CC1032ST1,CC0299ST,CC0802ST2,CC0302LIVER,CC0543LIVER,CC0772LIVER
0,Sample Type,Normal Lung,Normal Lung,KPT-E,KPT-E,KPT-E,KT,KT,KT,TMet,...,Soft Tissue Metastasis,Soft Tissue Metastasis,Soft Tissue Metastasis,Soft Tissue Metastasis,Soft Tissue Metastasis,Soft Tissue Metastasis,Soft Tissue Metastasis,Liver Metastasis,Liver Metastasis,Liver Metastasis
1,ENSMUSG00000089699,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.278725,0,0,0,0.178496
2,ENSMUSG00000090031,3.83325,3.86087,1.45903,4.29334,1.74992,1.16075,2.28923,3.31627,2.1086,...,4.08941,1.70325,1.55167,5.60458,3.55647,0.72832,2.06729,5.51027,1.93126,0.738178
3,ENSMUSG00000097797,7.65207,7.41595,2.9648,3.84807,1.91583,2.43193,2.4053,2.88517,3.44249,...,2.26774,1.85569,2.27174,3.24262,2.7234,2.55147,2.27523,3.71971,2.97135,1.79607
4,ENSMUSG00000085623,8.13192,7.84911,3.74511,5.64899,2.59978,4.0636,3.45774,4.30228,5.68482,...,3.84327,2.40616,2.72614,4.43781,3.71288,3.38767,2.69275,4.36387,3.31407,2.2306


In [ ]:
sample_types = supplementary_table_1_nature.iloc[0, 1:]  

# Extract gene expression data and set gene names as index
data = supplementary_table_1_nature.iloc[1:, :]
data.columns = ['Gene'] + sample_types.tolist()
data = data.set_index('Gene')

# Convert all values to numeric (in case they are strings)
data = data.apply(pd.to_numeric)

# Group columns by sample type and average across each group
averaged_df = data.groupby(level=0, axis=1).mean()

# Reset index if needed
averaged_df = averaged_df.reset_index()

In [5]:
averaged_df.head()

,Gene,KPT-E,KT,Liver Metastasis,Lymph Node Metastasis,Normal Lung,Pleural DTCs,Pleural Metastasis,Soft Tissue Metastasis,TMet,TnonMet
0,ENSMUSG00000089699,0.000000,0.000000,0.059499,0.000000,0.000000,0.000000,0.000000,0.034841,0.000000,0.000000
1,ENSMUSG00000090031,2.500763,2.255417,2.726569,2.200771,3.847060,1.072225,1.798785,2.691225,3.071133,3.238207
2,ENSMUSG00000097797,2.909567,2.574133,2.829043,3.151794,7.534010,2.413670,2.725507,2.536139,3.579558,3.702359
3,ENSMUSG00000085623,3.997960,3.941207,3.302847,4.438364,7.990515,3.735100,3.991648,3.453247,5.908950,5.815235
4,ENSMUSG00000079671,0.534196,0.750481,0.516575,0.962546,5.886478,6.065057,0.792822,1.083891,1.163125,0.769702


In [6]:
mouse_genes = averaged_df['Gene'].tolist()  # Get ENSMUSG IDs from index

mg = mygene.MyGeneInfo()
orthologs = mg.querymany(mouse_genes, 
                        scopes='ensembl.gene',
                        fields='symbol,homologene',
                        species='mouse')

gene_mapping = {}
no_ortholog = []

for item in orthologs:
    mouse_id = item['query']
    
    if 'homologene' in item and item['homologene']:
        homologene_data = item['homologene']
        if isinstance(homologene_data, dict) and 'genes' in homologene_data:
            genes = homologene_data['genes']
            
            for gene_info in genes:
                if isinstance(gene_info, dict) and gene_info.get('taxid') == 9606:
                    gene_mapping[mouse_id] = gene_info.get('symbol', 'Unknown')
                    break
        
        if mouse_id not in gene_mapping and 'symbol' in item:
            gene_mapping[mouse_id] = item['symbol']
    
    if mouse_id not in gene_mapping:
        no_ortholog.append(mouse_id)

print(f"Successfully mapped: {len(gene_mapping)} genes")
print(f"No mapping found for: {len(no_ortholog)} genes")

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('ENSMUSG00000072694', 2)]
1057 input query terms found no hit:	['ENSMUSG00000101738', 'ENSMUSG00000097109', 'ENSMUSG00000100371', 'ENSMUSG00000100237', 'ENSMUSG000


Successfully mapped: 19873 genes
No mapping found for: 7909 genes


In [11]:
averaged_df['Human_Ortholog'] = averaged_df['Gene'].map(gene_mapping)
averaged_df = averaged_df.dropna(subset=['Human_Ortholog'])

averaged_df.head()

averaged_df.to_csv('../data/averaged_gene_expression_nature_mice_supp_1.csv', index=False)